In [1]:
import pathlib, re, numpy as np, pandas as pd

# ── 0) Paths & simple config ──────────────────────────────────────────────
ROOT_DIR  = pathlib.Path(r"C:\Repositories\odi-data-visualization")
DATA_FILE = ROOT_DIR / "data" / "table_3.csv"
OUT_CSV_DEPTS = ROOT_DIR / "top_bottom_departments.csv"
OUT_CSV_GROUP = ROOT_DIR / "top_bottom_groups.csv"
OUT_XLSX      = ROOT_DIR / "top_bottom_summary.xlsx"

# Exclude metric names that match these regexes (case-insensitive).
EXCLUDE_METRIC_PATTERNS = [r"(?i)^scale\s*score$"]   # add more if needed, or set [] to keep all

# Map depts to Elected Officials (everything else -> Mayor’s Office)
ELECTED_DEPTS = {
    "the city auditor","city auditor",
    "city attorney",
    "city council",
    "treasurers office","treasurer's office","treasurer office"
}

def group_of(dept: str) -> str:
    return "Elected Officials" if dept.casefold().strip() in ELECTED_DEPTS else "Mayor’s Office"

# ── 1) Load & tidy table ──────────────────────────────────────────────────
df = pd.read_csv(DATA_FILE)

metric_col = df.columns[0]
df = df.rename(columns={metric_col: "Metric"})
departments = df.columns[1:].tolist()

# Coerce numeric columns
for c in departments:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Optional: drop excluded metrics
if EXCLUDE_METRIC_PATTERNS:
    mask = pd.Series(True, index=df.index)
    for pat in EXCLUDE_METRIC_PATTERNS:
        mask &= ~df["Metric"].str.contains(pat, regex=True, na=False)
    df = df[mask]

# Long format
long = (df
        .melt(id_vars="Metric", var_name="Department", value_name="Score")
        .dropna(subset=["Score"]))

# ── 2) Top/Bottom 3 for each department ───────────────────────────────────
def top_n_by_group(ddf, n=3):
    return (ddf.sort_values(["Department","Score"], ascending=[True, False])
               .groupby("Department", as_index=False)
               .head(n)
               .assign(RankType=f"Top {n}"))

def bottom_n_by_group(ddf, n=3):
    return (ddf.sort_values(["Department","Score"], ascending=[True, True])
               .groupby("Department", as_index=False)
               .head(n)
               .assign(RankType=f"Bottom {n}"))

dept_top3    = top_n_by_group(long, 3)
dept_bottom3 = bottom_n_by_group(long, 3)

dept_summary = (pd.concat([dept_top3, dept_bottom3], ignore_index=True)
                  .sort_values(["Department","RankType","Score"],
                               ascending=[True, True, False]))

# ── 3) Group averages (Mayor’s Office vs Elected Officials) ───────────────
long["Group"] = long["Department"].apply(group_of)

group_means = (long.groupby(["Group","Metric"], as_index=False)["Score"].mean())

grp_top3    = (group_means.sort_values(["Group","Score"], ascending=[True, False])
                         .groupby("Group", as_index=False)
                         .head(3)
                         .assign(RankType="Top 3"))

grp_bottom3 = (group_means.sort_values(["Group","Score"], ascending=[True, True])
                           .groupby("Group", as_index=False)
                           .head(3)
                           .assign(RankType="Bottom 3"))

group_summary = (pd.concat([grp_top3, grp_bottom3], ignore_index=True)
                   .sort_values(["Group","RankType","Score"],
                                ascending=[True, True, False]))

# ── 4) Save outputs ───────────────────────────────────────────────────────
dept_summary.to_csv(OUT_CSV_DEPTS, index=False)
group_summary.to_csv(OUT_CSV_GROUP, index=False)

with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xw:
    dept_summary.to_excel(xw, sheet_name="Departments", index=False)
    group_summary.to_excel(xw, sheet_name="Groups", index=False)
    # optional: autosize columns (xlsxwriter)
    for sheet, df_ in [("Departments", dept_summary), ("Groups", group_summary)]:
        ws = xw.sheets[sheet]
        for i, col in enumerate(df_.columns, start=1):
            width = max(12, min(60, int(df_[col].astype(str).str.len().max()) + 2))
            ws.set_column(i-1, i-1, width)

print("✓ Saved:")
print("   -", OUT_CSV_DEPTS.name)
print("   -", OUT_CSV_GROUP.name)
print("   -", OUT_XLSX.name)

# Quick peek
display(dept_summary.head(12))
display(group_summary)


✓ Saved:
   - top_bottom_departments.csv
   - top_bottom_groups.csv
   - top_bottom_summary.xlsx


,Metric,Department,Score,RankType
74,Inter-Department Communication,Board of Health,3.17,Bottom 3
73,Communication with Upper Management,Board of Health,2.83,Bottom 3
72,Innovation,Board of Health,2.58,Bottom 3
0,Task Significance,Board of Health,4.44,Top 3
1,Autonomy,Board of Health,4.08,Top 3
2,Supervisor Clarity/ Communication,Board of Health,4.00,Top 3
77,Communication with Upper Management,Building and Zoning Services,3.53,Bottom 3
76,Inter-Department Communication,Building and Zoning Services,3.20,Bottom 3
75,Innovation,Building and Zoning Services,2.97,Bottom 3
3,Supervisor Clarity/ Communication,Building and Zoning Services,4.31,Top 3


,Group,Metric,Score,RankType
8,Elected Officials,Communication with Upper Management,3.6100,Bottom 3
7,Elected Officials,Innovation,3.3150,Bottom 3
6,Elected Officials,Inter-Department Communication,3.1900,Bottom 3
0,Elected Officials,Task Significance,4.4650,Top 3
1,Elected Officials,Work-life Balance,4.3150,Top 3
2,Elected Officials,Satisfaction with the Job Itself,4.2650,Top 3
11,Mayor’s Office,Communication with Upper Management,3.4905,Bottom 3
10,Mayor’s Office,Inter-Department Communication,3.1290,Bottom 3
9,Mayor’s Office,Innovation,2.9705,Bottom 3
3,Mayor’s Office,Task Significance,4.3200,Top 3
